In [ ]:
from secret_sauce import *
from lines_code import *
from my_widgets import *
from ipywidgets import fixed, interact
%matplotlib inline

"""
We are going to use two example images to test our code on! 

TO DO: 
 - Hold down 'SHIFT' and press 'ENTER' to see the images
"""

show_images(white_lines, yellow_lines)

In [ ]:
"""
Let's find color thresholds! 

TO DO: 
 - Hold down 'SHIFT' and press 'ENTER' again.
"""
w = interact(show_color_threshold_results, red=red_widget, blue=blue_widget, green=green_widget)

In [ ]:
"""
Let's find Canny edge thresholds! 

TO DO: 
 - Hold down 'SHIFT' and press 'ENTER' again.
"""
w = interact(show_edge_detection_results, low_threshold=low_widget, high_threshold=high_widget)

In [ ]:
"""
Now we are going to put these thresholds together in a function!

TO DO:
 - Enter the numbers you got in the first two sections into the python code below
 - Then hold down 'SHIFT' and press 'ENTER' to see the results
 - To try different numbers, edit again, and press SHIFT+ENTER again!
"""

def threshold_image(image):
    
    """ ------------- START EDITING HERE ------------- """
    low_red_threshold = ?
    high_red_threshold = ?

    low_blue_threshold = ?
    high_blue_threshold = ?

    low_green_threshold = ?
    high_green_threshold = ?

    low_edge_threshold = ?
    high_edge_threshold = ?
    """ ------------- STOP EDITING HERE ------------- """

    # The 'apply_masks' function will take in an image (for example, 'white_lines'), and output the
    # result of apply the color and edge masks!
    return apply_masks(image, 
                       low_red_threshold, high_red_threshold, 
                       low_blue_threshold, high_blue_threshold,
                       low_green_threshold, high_green_threshold, 
                       low_edge_threshold, high_edge_threshold)

show_images(white_lines, threshold_image(white_lines), yellow_lines, threshold_image(yellow_lines))

In [ ]:
"""
We only look at a small part of the image to find lane lines - no need to look for lane lines in the sky!
We call this the "REGION OF INTEREST"

Let's find good numbers to define this!

TO DO:
 - Hold down 'SHIFT' and press 'ENTER' to see the region in which we look for lane lines
"""
w = interact(show_regions_of_interest, 
             top_height=height_widget, 
             bottom_width=bottom_width_widget, 
             top_width=top_width_widget)

In [ ]:
"""
Let's write a function to apply remove everything outside of our ROI from an image

TO DO:
 - Enter the numbers you got in the previous section into the python code below
 - Then hold down 'SHIFT' and press 'ENTER' to see the results
 - To try different numbers, edit again, and press SHIFT+ENTER again!
"""

def region_of_interest(image):
    """ ------------- START EDITING HERE ------------- """
    top_height = ?
    bottom_width = ?
    top_width = ?
    """ ------------- STOP EDITING HERE ------------- """
    
    vertices = get_vertices(top_height, bottom_width, top_width)
    
    return mask_roi(image, vertices)

white_masked_image = region_of_interest(threshold_image(white_lines))
yellow_masked_image = region_of_interest(threshold_image(yellow_lines))

show_images(white_lines, white_masked_image, yellow_lines, yellow_masked_image)

In [ ]:
"""
Let's find thresholds for hough lines! 

TO DO: 
 - Hold down 'SHIFT' and press 'ENTER' again.
"""

w = interact(show_hough_lines, masked_image1=fixed(white_masked_image), masked_image2=fixed(yellow_masked_image),
            min_line_length=min_line_widget, max_line_gap=max_line_widget)

In [ ]:
"""
Now we will write a function to draw lines on an image

TO DO:
 - Fill in 'min_line_length' with the values you found in "Minimum Pixels for a Line"
 - Fill in 'max_line_gap' with the values you found in "Maximum Gap in a Line"
 - Fill in 'color' with any color, represented as three numbers in parentheses
     - For example, 'color = (255, 0, 0)'
     - Some color codes are:
         * red : (255, 0, 0)
         * blue: (0, 255, 0)
         * green: (0, 0, 255)
         * magenta: (255, 0, 255)
         * light blue: (0, 255, 255)
         * purple: (80, 20, 255)
         * teal: (0, 255, 70)
 - Fill in 'thickness' with how thick you want your lane lines to be drawn, in number of pixels
     (We recommend around 5-10)
 - Hold down 'SHIFT' and press 'ENTER' again to see lane lines drawn on the images
"""

def draw_lines(original_image, masked_image):
    # Now we combine lines from our HOUGH results above to get two lines corresponding to the lane lines
    # Insert the thresholds you found in the next few lines!
    
    """ ------------- START EDITING HERE ------------- """
    min_line_length = ?
    max_line_gap = ?
    
    color = (?, ?, ?)
    
    thickness = ?
    """ ------------- STOP EDITING HERE ------------- """

    # We get the hough lines using 'get_hough_lines'
    hough_lines = get_hough_lines(masked_image, min_line_length, max_line_gap)

    # Then we've written a function to find the two lane lines from these lines
    lines = get_lane_lines(hough_lines, masked_image.shape)
    
    line_image = np.copy(original_image) * 0
    for l in lines:
        cv2.line(line_image, (l[0][0], l[0][1]), (l[0][2], l[0][3]), color, thickness)
    return cv2.addWeighted(line_image, 2.0, original_image, 0.7, 0)

show_images(white_lines, draw_lines(white_lines, white_masked_image),
           yellow_lines, draw_lines(yellow_lines, yellow_masked_image))

In [ ]:
"""
Combine the functions we've written so far to make one easy-to-use function to get lane lines.

TO DO: 
 - Hold down 'SHIFT' and press 'ENTER' to double check your results!
"""
def pipeline(image):
    
    
    # Use our function to get the color and edge thresholded image
    thresholded_image = threshold_image(image)
    
    # Next, use our function to mask the thresholded_image based on the region of interest (ROI)
    masked_image = region_of_interest(thresholded_image)
    
    # Next, use our function to draw the lane lines on the image!
    final_result = draw_lines(image, masked_image)
    

    return final_result

show_images(white_lines, pipeline(white_lines),
           yellow_lines, pipeline(yellow_lines))

In [ ]:
"""
Now we will run your code on a video to get the lane lines over time!

TO DO: 
 - Hold down 'SHIFT' and press 'ENTER' to create your video
"""
from moviepy.editor import VideoFileClip
from IPython.display import HTML
output = 'white.mp4'
clip = VideoFileClip("solidWhiteRight.mp4")
white_clip = clip.fl_image(pipeline)
%time white_clip.write_videofile(output, audio=False)

In [ ]:
"""
Play the video!

TO DO: 
 - Wait for the above section to be done (you'll see the line '[MoviePy] Done.')
 - Hold down 'SHIFT' and press 'ENTER' to play your video
"""
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(output))